<a href="https://colab.research.google.com/github/HelmiAlf/Projects/blob/main/%5BCV%5D_Improving_state_of_the_art_method_on_Image_Recognition_Architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/Colab Notebooks

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks


In [ ]:
path = 'Face from Different Age, Gender and Ethnicity/'
img_path = 'Face from Different Age, Gender and Ethnicity/images/'

In [ ]:
def evaluate(y_test, y_pred): 
    print("* Confusion Matrix") 
    print(pd.DataFrame(confusion_matrix(y_test, y_pred), \
            index = ['Actual Class ' + str(i) for i in range(len(np.unique(y_test)))], \
            columns = ['Pred Class ' + str(i) for i in range(len(np.unique(y_test)))] )) 
      
    print ("\n* Accuracy : ", 
    accuracy_score(y_test,y_pred)*100) 
      
    print("\n* Classification Report :\n", 
    classification_report(y_test, y_pred, zero_division=0)) 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report 


import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dense
from tensorflow.keras.layers import AvgPool2D, GlobalAveragePooling2D, MaxPool2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ReLU, concatenate, add
import tensorflow.keras.backend as K

In [ ]:
data = pd.read_csv(path+'Face.csv')
gender = data['gender']
age = data['age']
ethn = data['ethnicity']

In [ ]:
image_arrays = []

df = pd.read_csv(path+'dataset_array.csv', header=None)

for i in range(df.shape[0]):
  arr = df.iloc[i] / 255
  two_d_arr = arr.to_numpy().reshape(48,48)
  image_arrays.append(two_d_arr)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(image_arrays, gender, test_size=0.2, random_state=42)
X_train = np.array(X_train)
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1], X_train.shape[2], 1)
X_test = np.array(X_test)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1], X_test.shape[2], 1)

## Visualization


In [ ]:
# gender
gender_labels = ['Male', 'Female']
n_class = len(gender_labels)

container = []
for i in range(n_class):
  container.append(np.zeros(image_arrays[0].shape))

for a in range(len(image_arrays)):
  gender_class = gender[a]
  container[gender_class] += image_arrays[a]

plt.figure(figsize=(10,3))

for n in range(n_class):
  plt.subplot(1, n_class, n+1)
  plt.imshow(container[n])
  plt.title('Average '+ gender_labels[n])

# plt.tight_layout(pad=1.0)
plt.show()

In [ ]:
# Ethn
ethn_labels = ['White', 'Black', 'Asian', 'Indian', 'Others']
n_class = len(ethn_labels)

container = []
for i in range(n_class):
  container.append(np.zeros(image_arrays[0].shape))

for a in range(len(image_arrays)):
  ethn_class = ethn[a]
  container[ethn_class] += image_arrays[a]

plt.figure(figsize=(10,3))

for n in range(n_class):
  plt.subplot(1, n_class, n+1)
  plt.imshow(container[n])
  plt.title('Average '+ ethn_labels[n])

# plt.tight_layout(pad=1.0)
plt.show()

In [ ]:
# age
age_range = age.max() - age.min()
bins = 10
n_class = age_range // bins + 1
# print(n_class)

age_container = []
for i in range(n_class):
  age_container.append(np.zeros(image_arrays[0].shape))

print(len(age_container))

for a in range(len(image_arrays)):
  age_class = (age[a]-age.min()) // bins
  age_container[age_class] += image_arrays[a]

plt.figure(figsize=(20,6))

for n in range(n_class):
  plt.subplot(1, n_class, n+1)
  plt.imshow(age_container[n])
  plt.title('range '+ str(age.min()*n*bins)+"-"+str(age.min()*bins*(n+1)))

plt.tight_layout(pad=3.0)
plt.show()

In [ ]:
# gender + ethn
labels = [
          ['Male', 'Female'],
          ['White', 'Black', 'Asian', 'Indian', 'Others ']
         ]

m = 2
n = 5

container = []

for i in range(m):
  container.append([])
  for j in range(n):
    container[i].append(np.zeros(image_arrays[0].shape))
    
for a in range(len(image_arrays)):
  i = gender[a]
  j = ethn[a]
  container[i][j] += image_arrays[a]

plt.figure(figsize=(20,6))

for i in range(m):
  for j in range(n):
    plt.subplot(m,n,(i*n)+j+1)
    plt.imshow(container[i][j])
    plt.title('average '+ labels[0][i]+" "+labels[1][j])

plt.tight_layout(pad=3.0)
plt.show()

In [ ]:
# gender + age
age_range = age.max() - age.min()
bins = 10
n_class = age_range // bins + 1

gender_labels = ['M', 'F']

container = []
for m in range(len(gender_labels)):
  container.append([])
  for n in range(n_class):
    container[m].append(np.zeros(image_arrays[0].shape))

# print(len(container))
# print(len(container[]))
for a in range(len(image_arrays)):
  age_class = (age[a]-age.min()) // bins
  gender_class = gender[a]
  container[gender_class][age_class] += image_arrays[a]

plt.figure(figsize=(20,6))

for m in range(len(gender_labels)):
  for n in range(n_class):
    plt.subplot(2, n_class, (m*n_class)+n+1)

    plt.imshow(container[m][n])
    plt.title(gender_labels[m]+' range '+ str(age.min()*n*bins)+"-"+str(age.min()*bins*(n+1)))

plt.tight_layout(pad=1.0)
plt.show()

In [ ]:
# ethn + age
age_range = age.max() - age.min()
bins = 10
n_class = age_range // bins + 1

ethn_labels = ['White', 'Black', 'Asian', 'Indian', 'Others']

container = []
for m in range(len(ethn_labels)):
  container.append([])
  for n in range(n_class):
    container[m].append(np.zeros(image_arrays[0].shape))

for a in range(len(image_arrays)):
  age_class = (age[a]-age.min()) // bins
  ethn_class = ethn[a]
  container[ethn_class][age_class] += image_arrays[a]

plt.figure(figsize=(20,6))

for m in range(len(ethn_labels)):
  for n in range(n_class):
    # print(m, " ", n, " ", (m*n_class)+n+1)
    plt.subplot(len(ethn_labels), n_class, (m*n_class)+n+1)

    plt.imshow(container[m][n])
    plt.title(ethn_labels[m]+' '+ str(age.min()*n*bins)+"-"+str(age.min()*bins*(n+1)))

plt.tight_layout(pad=1.0)
plt.show()

## Modelling

### Densenet121

In [ ]:
# Densenet121
def densenet(input_shape, n_classes, filters = 32):
    
    #batch norm + relu + conv
    def bn_rl_conv(x,filters,kernel=1,strides=1):
        
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(filters, kernel, strides=strides,padding = 'same')(x)
        return x
    
    def dense_block(x, repetition):
        
        for _ in range(repetition):
            y = bn_rl_conv(x, 4*filters)
            y = bn_rl_conv(y, filters, 3)
            x = concatenate([y,x])
        return x
        
    def transition_layer(x):
        
        x = bn_rl_conv(x, K.int_shape(x)[-1] //2 )
        x = AvgPool2D(2, strides = 2, padding = 'same')(x)
        return x
    
    input = Input (input_shape)
    x = Conv2D(64, 7, strides = 2, padding = 'same')(input)
    x = MaxPool2D(3, strides = 2, padding = 'same')(x)
    
    for repetition in [6,12,24,16]:
        
        d = dense_block(x, repetition)
        x = transition_layer(d)
    x = GlobalAveragePooling2D()(d)
    output = Dense(n_classes, activation = 'softmax')(x)
    
    model = Model(input, output)
    return model


In [ ]:
input_shape = 48, 48, 1
n_classes = 2
model = densenet(input_shape,n_classes)
model.summary()

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=10)

In [ ]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

evaluate(y_test, y_pred)

### ModifiedDenseNet

In [ ]:
def modified_densenet(input_shape, n_classes, filters = 32):
    
    #batch norm + relu + conv
    def bn_rl_conv(x,filters,kernel=1,strides=1):
        
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(filters, kernel, strides=strides,padding = 'same')(x)
        return x
    
    def dense_block(x, repetition):
        
        for _ in range(repetition):
            y = bn_rl_conv(x, 4*filters) # modified
            y = bn_rl_conv(y, 2*filters, 2) # new
            y = bn_rl_conv(y, filters, 3) 
            x = concatenate([y,x])
        return x
        
    def transition_layer(x):
        
        x = bn_rl_conv(x, K.int_shape(x)[-1] //2 )
        x = AvgPool2D(2, strides = 2, padding = 'same')(x)
        return x
    
    input = Input (input_shape)
    x = Conv2D(64, 7, strides = 2, padding = 'same')(input)
    x = MaxPool2D(3, strides = 2, padding = 'same')(x)
    
    for repetition in [6,8,8,6]: # modified
        
      d = dense_block(x, repetition)
      x = transition_layer(d)
    x = GlobalAveragePooling2D()(d)
    output = Dense(n_classes, activation = 'softmax')(x)
    
    model = Model(input, output)
    return model


In [ ]:
input_shape = 48, 48, 1
n_classes = 2
model = modified_densenet(input_shape,n_classes, filters = 24)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 48, 48, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 24, 24, 64)   3200        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 12, 12, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 12, 12, 64)   256         max_pooling2d[0][0]              
______________________________________________________________________________________________

In [ ]:
history = model.fit(X_train, y_train, epochs=10)

Epoch 1/10
475/475 [==============================] - 288s 606ms/step - loss: 0.5340 - accuracy: 0.7410
Epoch 2/10
475/475 [==============================] - 282s 594ms/step - loss: 0.3821 - accuracy: 0.8251
Epoch 3/10
475/475 [==============================] - 285s 601ms/step - loss: 0.3339 - accuracy: 0.8551
Epoch 4/10
475/475 [==============================] - 286s 601ms/step - loss: 0.3085 - accuracy: 0.8664
Epoch 5/10
475/475 [==============================] - 281s 591ms/step - loss: 0.2849 - accuracy: 0.8785
Epoch 6/10
475/475 [==============================] - 284s 599ms/step - loss: 0.2631 - accuracy: 0.8875
Epoch 7/10
475/475 [==============================] - 286s 602ms/step - loss: 0.2502 - accuracy: 0.8958
Epoch 8/10
475/475 [==============================] - 286s 602ms/step - loss: 0.2359 - accuracy: 0.9034
Epoch 9/10
475/475 [==============================] - 285s 599ms/step - loss: 0.2178 - accuracy: 0.9098
Epoch 10/10
475/475 [==============================] - 285s 601m

In [ ]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

evaluate(y_test, y_pred)